In [1]:
import pandas as pd

url = 'https://raw.githubusercontent.com/JeanFraga/DS8-Build_Week-1/master/notebooks/Restaurant_Consumer_Data_merged'

df = pd.read_csv(url)
print(df.shape)
df.head()

(1161, 116)


,userid,placeid,rating,food_rating,service_rating,ulatitude,ulongitude,smoker,drink_level,dress_preference,...,rcuisine_seafood,rcuisine_steaks,rcuisine_sushi,rcuisine_vegetarian,rtotal_cuisine,accepts_amex,accepts_debit,accepts_cash,accepts_mastercard,accepts_visa
0,u1077,135085,2,2,2,22.156469,-100.985540,False,social drinker,elegant,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
1,u1077,135038,2,2,1,22.156469,-100.985540,False,social drinker,elegant,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,u1077,132825,2,2,2,22.156469,-100.985540,False,social drinker,elegant,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,u1077,135060,1,2,2,22.156469,-100.985540,False,social drinker,elegant,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,u1068,135104,1,1,2,23.752269,-99.168605,False,casual drinker,informal,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0


In [2]:
from sklearn.model_selection import train_test_split

target1 = 'rating'
target2 = 'food_rating'
target3 = 'service_rating'

X = df.drop(columns=[target1, target2, target3])
y1 = df[target1]
y2 = df[target2]
y3 = df[target3]

X_train, X_test,y1_train, y1_test,y2_train, y2_test,y3_train, y3_test= train_test_split(X,y1,y2,y3, test_size=0.2, random_state=7)
X_train.shape, X_test.shape,y1_train.shape, y1_test.shape,y2_train.shape, y2_test.shape,y3_train.shape, y3_test.shape

((928, 113), (233, 113), (928,), (233,), (928,), (233,), (928,), (233,))

In [3]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report

dummy = DummyClassifier(strategy= 'stratified',random_state=7)
y1_pred = dummy.fit(X_train, y1_train).predict(y1_test)
print(y1_train.value_counts(normalize=True))
print(classification_report(y1_test, y1_pred))

2    0.399784
1    0.375000
0    0.225216
Name: rating, dtype: float64
              precision    recall  f1-score   support

           0       0.22      0.29      0.25        45
           1       0.31      0.36      0.33        73
           2       0.47      0.37      0.41       115

    accuracy                           0.35       233
   macro avg       0.33      0.34      0.33       233
weighted avg       0.37      0.35      0.35       233



In [4]:
dummy = DummyClassifier(strategy= 'stratified',random_state=7)
y2_pred = dummy.fit(X_train, y2_train).predict(y2_test)
print(y2_train.value_counts(normalize=True))
print(classification_report(y2_test, y2_pred))

2    0.434267
1    0.334052
0    0.231681
Name: food_rating, dtype: float64
              precision    recall  f1-score   support

           0       0.15      0.18      0.16        51
           1       0.36      0.39      0.38        69
           2       0.48      0.42      0.45       113

    accuracy                           0.36       233
   macro avg       0.33      0.33      0.33       233
weighted avg       0.37      0.36      0.36       233



In [5]:
dummy = DummyClassifier(strategy= 'stratified',random_state=7)
y3_pred = dummy.fit(X_train, y3_train).predict(y3_test)
print(y3_train.value_counts(normalize=True))
print(classification_report(y3_test, y3_pred))

1    0.372845
2    0.351293
0    0.275862
Name: service_rating, dtype: float64
              precision    recall  f1-score   support

           0       0.21      0.25      0.23        59
           1       0.32      0.34      0.33        80
           2       0.37      0.31      0.34        94

    accuracy                           0.30       233
   macro avg       0.30      0.30      0.30       233
weighted avg       0.31      0.30      0.31       233



In [14]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import category_encoders as ce
from sklearn.pipeline import make_pipeline

model = XGBClassifier(
    n_estimators=1000,
    max_depth=2,
    learning_rate=0.1,
    n_jobs=-1,
    tree_method = 'exact'
)

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    model
)

cross_val_score(pipeline, X_train, y1_train, 
                scoring='accuracy', cv=10, n_jobs=-1)

array([0.54255319, 0.64516129, 0.50537634, 0.6344086 , 0.62365591,
       0.53763441, 0.51612903, 0.61290323, 0.61956522, 0.52747253])

In [15]:
cross_val_score(pipeline, X_train, y2_train, 
                scoring='accuracy', cv=10, n_jobs=-1)

array([0.56382979, 0.65957447, 0.55319149, 0.64516129, 0.5483871 ,
       0.54347826, 0.52173913, 0.52173913, 0.56521739, 0.56521739])

In [16]:
cross_val_score(pipeline, X_train, y3_train, 
                scoring='accuracy', cv=10, n_jobs=-1)

array([0.61702128, 0.61702128, 0.43617021, 0.54255319, 0.54255319,
       0.4787234 , 0.51648352, 0.49450549, 0.52747253, 0.59340659])

In [9]:
pipeline

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=None, drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=None, return_df=True, verbose=0)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=0,
                               min_child_weight=1, missing=None,
                               n_estimators=1000, n_jobs=-1, nthread=None,
                               objective='binary:logistic', random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
  

In [67]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
#from sklearn.cross_validation import *

model = XGBClassifier(
#     n_estimators=1000,
#     max_depth=2,
#     learning_rate=0.1,
#     n_jobs=-1,
#     tree_method = 'exact'
)

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    model
)

# cross_val_score(pipeline, X_train, y1_train, 
#                 scoring='accuracy', cv=10, n_jobs=-1)

xgb_model = XGBClassifier()
parameters = {'xgbclassifier__nthread':[8],
              #'xgbclassifier__objective':['binary:logistic'],
              'xgbclassifier__learning_rate': [.1, .05], 
              'xgbclassifier__max_depth': range(2,10),
              'xgbclassifier__min_child_weight': range(1,5),
              #'xgbclassifier__silent': [1],
              'xgbclassifier__subsample': [0.5, 0.7],
              'xgbclassifier__colsample_bytree': [0.5, 0.7],
              'xgbclassifier__n_estimators': [1000], 
              'xgbclassifier__tree_method': ['exact', 'hist']
              #'xgbclassifier__seed': [1337, 1]
             }

clf = GridSearchCV(pipeline, parameters, n_jobs=-1, 
                   cv=StratifiedKFold(n_splits=5, shuffle=True), 
                   scoring='accuracy',
                   verbose=2, refit=True,return_train_score=True)
clf.fit(X_train, y1_train)

Fitting 5 folds for each of 512 candidates, totalling 2560 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   55.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 19.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 56.3min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 83.5min
exception calling callback for <Future at 0x2023f8b2248 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "C:\ProgramData\Anaconda3\lib\site-packages\joblib\parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iter

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [ ]:
from sklearn.externals import joblib
filename = 'xgboost_model_y1.joblib'
joblib.dump(clf, filename)

In [19]:
from sklearn.model_selection import GridSearchCV

In [65]:
best_parameters, score, _ = max(clf.cv_results_, key=lambda x: x[1])
#print('Accuracy score:', score)
# for param_name in sorted(best_parameters.keys()):
#     print("%s: %r" % (param_name, best_parameters[param_name]))

ValueError: too many values to unpack (expected 3)

In [64]:
max(clf.cv_results_.keys())

'std_train_score'

In [66]:
clf.cv_results_

{'mean_fit_time': array([ 4.11798437,  7.84701029,  3.27091726,  6.90719128,  3.06779416,
         7.14522099,  3.24565188,  7.0115788 ,  4.21605611, 11.92585945,
         4.45389016, 12.5400184 ,  5.64041416, 13.48715083,  6.86508632,
        13.41767772,  7.18790809, 14.69825943,  5.90959485, 15.19778339,
         5.70496321, 14.02320854,  5.92522065, 12.585639  ,  6.14246647,
        14.64738369,  5.5530258 , 14.39487131,  5.85199277, 13.62897698,
         5.27275395, 13.61061486,  6.98875642, 18.12725123,  6.78014382,
        17.00705059,  7.26919516, 16.53895768,  7.23012829, 15.75850034,
         7.55693404, 18.43644889,  7.33268682, 17.30543105,  7.44224008,
        16.92344562,  7.35863018, 16.40813414,  7.98281741, 19.64674632,
         7.60647766, 16.69289207,  7.09530926, 16.7322588 ,  7.4417367 ,
        16.14447951,  7.63771613, 18.60764742,  7.0361348 , 17.7685558 ,
         7.47963858, 16.88917685,  7.51653028, 12.70786103]),
 'std_fit_time': array([0.00293609, 0.4210185

In [63]:
clf.get_params()

{'cv': StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
 'error_score': 'raise-deprecating',
 'estimator__memory': None,
 'estimator__steps': [('ordinalencoder',
   OrdinalEncoder(cols=None, drop_invariant=False, handle_missing='value',
                  handle_unknown='value', mapping=None, return_df=True, verbose=0)),
  ('xgbclassifier',
   XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                 colsample_bynode=1, colsample_bytree=1, gamma=0,
                 learning_rate=0.1, max_delta_step=0, max_depth=3,
                 min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
                 nthread=None, objective='binary:logistic', random_state=0,
                 reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                 silent=None, subsample=1, verbosity=1))],
 'estimator__verbose': False,
 'estimator__ordinalencoder': OrdinalEncoder(cols=None, drop_invariant=False, handle_missing='value',
                han

In [62]:
clf.score(X_test, y=y1_test)

0.6394849785407726